In [8]:
!pwd

/home/ubuntu/Notebook


In [9]:
!ls

diabetic-Copy1.ipynb  diabetic.ipynb


In [10]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import pandas as pd
import numpy as np
# from medpy.io import load
from matplotlib import pyplot as plt
import glob
import cv2

import sklearn.metrics as sklm

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model

import scipy
import matplotlib.image as mpimg
from scipy.misc import imread

%matplotlib inline

In [11]:
rootDir = "/home/ubuntu/Yang_Sahana/test"

In [ ]:
def CroppedImagev1(filename, CropRectangle = True):
    img = mpimg.imread(filename)
    height, width, channels = img.shape
    if CropRectangle:
        if width > height:
            delta = width - height
            left = int(delta/2)
            upper = 0
            right = height + left
            lower = height
        else:
            delta = height - width
            left = 0
            upper = int(delta/2)
            right = width
            lower = width + upper
        img = img[upper:lower, left:right]
    plt.imshow(img)
    print "final height, width", img.shape
    return img

In [32]:
def crop_borders(img_path, threshold=0.01):
    img = imread(img_path)
    if img.shape == ():
        return img

    rgb_added = img.sum(axis=2)
    mean_px = rgb_added.mean()
    std_px = rgb_added.std()
    # not using std as it can be very high

    col_sum = rgb_added.sum(axis=0)
    row_sum = rgb_added.sum(axis=1)

    thresh_val = (mean_px)*threshold
    col_index = np.where(col_sum > thresh_val*col_sum.shape[0])
    row_index = np.where(row_sum > thresh_val*row_sum.shape[0])
    hstart, hend = col_index[0][0], col_index[0][-1]
    wstart, wend = row_index[0][0], row_index[0][-1]
    return img[wstart:wend, hstart:hend]

In [13]:
resize_128_dir = "/home/ubuntu/final_data/test/128"
# Create directory for 128x128 resize
try:
    os.stat(resize_128_dir)
except:
    os.makedirs(resize_128_dir)
    
resize_256_dir = "/home/ubuntu/final_data/test/256"
# Create directory for 256x256 resize
try:
    os.stat(resize_256_dir)
except:
    os.makedirs(resize_256_dir)

resize_512_dir = "/home/ubuntu/final_data/test/512"
# Create directory for 512x512 resize
try:
    os.stat(resize_512_dir)
except:
    os.makedirs(resize_512_dir)

In [33]:
# Creating .data files

# Check if all the files are jpeg format
# if not, skip it
# else, apply crop border function on it
# when you convert to .data, the .jpeg is till there in the filename
# therefore, replace the .jpeg with ''

count = 0
for root, dirs, files in os.walk(rootDir):
    for fileName in files:
        if not fileName.endswith('.jpeg'):
            continue
        count = count + 1
        fileName_new = fileName.replace(".jpeg", "")
        fileName_256 = resize_256_dir + '/' + fileName_new + '.data'
        if os.path.isfile(fileName_256):
            continue
        #print "count = ", count, "filename = ", fileName
        img = crop_borders(rootDir + '/' + fileName)
        if img.shape != ():
            img_256 = cv2.resize(img, (256,256))
            img_256.tofile(fileName_256, sep="", format='uint8')


In [35]:
# Creating .data files

# Check if all the files are jpeg format
# if not, skip it
# else, apply crop border function on it
# when you convert to .data, the .jpeg is till there in the filename
# therefore, replace the .jpeg with ''

for root, dirs, files in os.walk(rootDir):
    for fileName in files:
        if not fileName.endswith('.jpeg'):
            continue
        fileName_new = fileName.replace(".jpeg", "")
        fileName_128 = resize_128_dir + '/' + fileName_new + '.data'
        fileName_512 = resize_512_dir + '/' + fileName_new + '.data'
        if os.path.isfile(fileName_128) and \
           os.path.isfile(fileName_512):
            continue
        img = crop_borders(rootDir + '/' + fileName)
        if img.shape != ():
            img_128 = cv2.resize(img, (128,128))
            img_128.tofile(fileName_128, sep="", format='uint8') # write it out as binary
            img_512 = cv2.resize(img, (512,512))
            img_512.tofile(fileName_512, sep="", format='uint8')